## 1. Import Libraries

In [1]:
# Data handling
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Optimization
from scipy.optimize import minimize
import cvxpy as cp

# Statistical analysis
from scipy import stats
from sklearn.covariance import LedoitWolf
from sklearn.cluster import KMeans

# Technical indicators
import ta

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

ModuleNotFoundError: No module named 'cvxpy'

## 2. Data Import & Preprocessing

### 2.1 Load All Datasets

In [ ]:
# Define file paths
data_folder = './PortfolioBuilding/'

files = {
    'BTC': 'BTC_USD Bitfinex Historical Data.csv',
    'ETH': 'ETH_USD Binance Historical Data.csv',
    'AAPL': 'Apple Stock Price History.csv',
    'AMZN': 'Amazon.com Stock Price History.csv',
    'MSFT': 'Microsoft Stock Price History.csv',
    'NVDA': 'NVIDIA Stock Price History.csv',
    'TSLA': 'Tesla Stock Price History.csv',
    'META': 'Meta Platforms Stock Price History.csv',
    'NASDAQ': 'Nasdaq 100 Historical Data.csv',
    'GOLD': 'Gold Futures Historical Data.csv',
    'SILVER': 'Silver Futures Historical Data.csv',
    'OIL': 'Crude Oil WTI Futures Historical Data.csv'
}

print("Loading datasets...")
raw_data = {}

for asset, filename in files.items():
    try:
        df = pd.read_csv(data_folder + filename)
        raw_data[asset] = df
        print(f"✓ {asset:8} - {len(df):5} rows")
    except Exception as e:
        print(f"✗ {asset:8} - Error: {e}")

print(f"\nTotal assets loaded: {len(raw_data)}")

### 2.2 Data Cleaning Function

In [ ]:
def clean_price_data(df, asset_name):
    """
    Clean and standardize price data from CSV files.
    - Convert Date to datetime
    - Clean Price column (remove commas, convert to float)
    - Handle missing values
    - Sort by date ascending
    """
    df_clean = df.copy()
    
    # Convert Date column
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%m/%d/%Y', errors='coerce')
    
    # Clean numeric columns (remove commas, convert to float)
    numeric_cols = ['Price', 'Open', 'High', 'Low']
    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.replace(',', '').astype(float)
    
    # Clean Volume column
    if 'Vol.' in df_clean.columns:
        df_clean['Volume'] = df_clean['Vol.'].astype(str).str.replace('K', '').str.replace('M', '').str.replace(',', '')
        df_clean['Volume'] = pd.to_numeric(df_clean['Volume'], errors='coerce')
    
    # Sort by date ascending
    df_clean = df_clean.sort_values('Date').reset_index(drop=True)
    
    # Handle missing values
    df_clean = df_clean.dropna(subset=['Date', 'Price'])
    
    # Forward fill remaining missing values
    df_clean = df_clean.fillna(method='ffill')
    
    print(f"{asset_name:8} - Cleaned: {len(df_clean)} rows, Date range: {df_clean['Date'].min().date()} to {df_clean['Date'].max().date()}\")")
    
    return df_clean

# Clean all datasets
cleaned_data = {}
for asset, df in raw_data.items():
    cleaned_data[asset] = clean_price_data(df, asset)

### 2.3 Align All Assets to Common Date Range

In [ ]:
# Create price matrix with all assets aligned
price_dict = {}
for asset, df in cleaned_data.items():
    price_dict[asset] = df.set_index('Date')['Price']

# Combine into single DataFrame
prices_df = pd.DataFrame(price_dict)

# Check for missing dates
print("Missing values before alignment:")
print(prices_df.isnull().sum())
print(f"\nDate range: {prices_df.index.min().date()} to {prices_df.index.max().date()}")
print(f"Total trading days: {len(prices_df)}")

# Forward fill missing values (for non-trading days)
prices_df = prices_df.fillna(method='ffill').fillna(method='bfill')

# Drop any remaining NaN rows
prices_df = prices_df.dropna()

print(f"\nAfter cleaning: {len(prices_df)} days with complete data")
print("\nFirst 5 rows:")
print(prices_df.head())

### 2.4 Calculate Returns

In [ ]:
# Calculate daily returns (percentage change)
returns_df = prices_df.pct_change().dropna()

# Calculate log returns for more accurate compounding
log_returns_df = np.log(prices_df / prices_df.shift(1)).dropna()

print("Daily Returns Statistics:")
print(returns_df.describe())
print(f"\nReturns shape: {returns_df.shape}")
print(f"Date range: {returns_df.index.min().date()} to {returns_df.index.max().date()}")

### 2.5 Visualize Price Data

In [ ]:
# Normalize prices to 100 for comparison
normalized_prices = (prices_df / prices_df.iloc[0]) * 100

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Crypto
axes[0].plot(normalized_prices[['BTC', 'ETH']])
axes[0].set_title('Cryptocurrency Performance (Normalized to 100)', fontsize=14, fontweight='bold')
axes[0].legend(['BTC', 'ETH'])
axes[0].set_ylabel('Normalized Price')
axes[0].grid(True, alpha=0.3)

# Equities
axes[1].plot(normalized_prices[['AAPL', 'AMZN', 'MSFT', 'NVDA', 'TSLA', 'META', 'NASDAQ']])
axes[1].set_title('Equity & Index Performance (Normalized to 100)', fontsize=14, fontweight='bold')
axes[1].legend(['AAPL', 'AMZN', 'MSFT', 'NVDA', 'TSLA', 'META', 'NASDAQ'], ncol=3)
axes[1].set_ylabel('Normalized Price')
axes[1].grid(True, alpha=0.3)

# Commodities
axes[2].plot(normalized_prices[['GOLD', 'SILVER', 'OIL']])
axes[2].set_title('Commodity Performance (Normalized to 100)', fontsize=14, fontweight='bold')
axes[2].legend(['GOLD', 'SILVER', 'OIL'])
axes[2].set_ylabel('Normalized Price')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.6 Return Distributions

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(18, 12))
axes = axes.flatten()

for idx, col in enumerate(returns_df.columns):
    axes[idx].hist(returns_df[col], bins=50, alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'{col} Returns', fontweight='bold')
    axes[idx].axvline(returns_df[col].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    axes[idx].set_xlabel('Daily Return')
    axes[idx].set_ylabel('Frequency')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Daily Return Distributions by Asset', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

### 2.7 Rolling Volatility

In [ ]:
# Calculate 30-day rolling volatility (annualized)
rolling_vol = returns_df.rolling(window=30).std() * np.sqrt(252)

fig, ax = plt.subplots(figsize=(15, 6))
for col in rolling_vol.columns:
    ax.plot(rolling_vol.index, rolling_vol[col], label=col, alpha=0.7)

ax.set_title('30-Day Rolling Volatility (Annualized)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Volatility')
ax.legend(ncol=4, loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nAverage Annualized Volatility by Asset:")
print((returns_df.std() * np.sqrt(252)).sort_values(ascending=False))

## 3. Feature Engineering

### 3.1 Correlation Analysis

In [ ]:
# Calculate correlation matrix
corr_matrix = returns_df.corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Asset Return Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Find highly correlated pairs
print("\nHighly Correlated Asset Pairs (|correlation| > 0.7):")
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            print(f"{corr_matrix.columns[i]:8} <-> {corr_matrix.columns[j]:8}: {corr_matrix.iloc[i, j]:.3f}")

### 3.2 Covariance Matrix (Ledoit-Wolf Shrinkage)

In [ ]:
# Use Ledoit-Wolf shrinkage for more stable covariance estimation
lw = LedoitWolf()
cov_matrix_lw = lw.fit(returns_df).covariance_

# Annualize covariance matrix
cov_matrix_annual = cov_matrix_lw * 252

# Convert to DataFrame
cov_df = pd.DataFrame(cov_matrix_annual, index=returns_df.columns, columns=returns_df.columns)

print("Annualized Covariance Matrix (Ledoit-Wolf):")
print(cov_df)

# Calculate annualized returns (mean)
annual_returns = returns_df.mean() * 252

print("\n\nAnnualized Expected Returns:")
print(annual_returns.sort_values(ascending=False))

### 3.3 Market Regime Detection (K-Means Clustering)

In [ ]:
# Create features for regime detection
regime_features = pd.DataFrame(index=returns_df.index)

# Rolling metrics (30-day windows)
regime_features['market_return'] = returns_df['NASDAQ'].rolling(30).mean()
regime_features['market_vol'] = returns_df['NASDAQ'].rolling(30).std()
regime_features['avg_correlation'] = returns_df.rolling(30).corr().groupby(level=0).mean().mean(axis=1)

# Remove NaN
regime_features = regime_features.dropna()

# Cluster into 3 regimes: Low Vol, Normal, High Vol
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
regime_features['regime'] = kmeans.fit_predict(regime_features[['market_return', 'market_vol', 'avg_correlation']])

# Label regimes based on volatility
regime_stats = regime_features.groupby('regime')['market_vol'].mean().sort_values()
regime_mapping = {regime_stats.index[0]: 'Low Volatility', 
                  regime_stats.index[1]: 'Normal', 
                  regime_stats.index[2]: 'High Volatility'}
regime_features['regime_label'] = regime_features['regime'].map(regime_mapping)

# Visualize regimes
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot regime over time
axes[0].scatter(regime_features.index, regime_features['market_vol'], 
                c=regime_features['regime'], cmap='viridis', alpha=0.5, s=10)
axes[0].set_title('Market Regimes Over Time', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Market Volatility')
axes[0].grid(True, alpha=0.3)

# Plot regime distribution
regime_counts = regime_features['regime_label'].value_counts()
axes[1].bar(regime_counts.index, regime_counts.values, color=['green', 'blue', 'red'])
axes[1].set_title('Regime Distribution', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Number of Days')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nRegime Statistics:")
print(regime_features.groupby('regime_label')[['market_return', 'market_vol', 'avg_correlation']].mean())

## 4. Portfolio Optimization Strategies

We implement three complementary optimization approaches:
1. **Mean-Variance Optimization** - Maximizes Sharpe ratio
2. **Risk Parity** - Equal risk contribution from each asset
3. **Regime-Adaptive Strategy** - Switches allocations based on market conditions

### 4.1 Mean-Variance Optimization (Markowitz)

**Mathematical Formulation:**

Maximize Sharpe Ratio:
$$\text{Sharpe} = \frac{w^T \mu - r_f}{\sqrt{w^T \Sigma w}}$$

Subject to:
- $\sum_{i=1}^{n} w_i = 1$ (full investment)
- $w_i \geq 0$ (no short selling)
- $0 \leq w_i \leq 0.3$ (position limits)

Where:
- $w$ = portfolio weights
- $\mu$ = expected returns
- $\Sigma$ = covariance matrix
- $r_f$ = risk-free rate (assumed 3%)

In [ ]:
def mean_variance_optimization(returns, cov_matrix, risk_free_rate=0.03, max_weight=0.30):
    """
    Mean-Variance Optimization to maximize Sharpe Ratio
    """
    n_assets = len(returns)
    
    # Define objective: minimize negative Sharpe ratio
    def neg_sharpe(weights):
        portfolio_return = np.dot(weights, returns)
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe = (portfolio_return - risk_free_rate) / portfolio_vol
        return -sharpe
    
    # Constraints
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})  # weights sum to 1
    
    # Bounds: 0 <= weight <= max_weight
    bounds = tuple((0, max_weight) for _ in range(n_assets))
    
    # Initial guess: equal weights
    initial_weights = np.array([1/n_assets] * n_assets)
    
    # Optimize
    result = minimize(neg_sharpe, initial_weights, method='SLSQP', 
                     bounds=bounds, constraints=constraints, options={'maxiter': 1000})
    
    return result.x

# Calculate optimal weights
mvo_weights = mean_variance_optimization(annual_returns.values, cov_matrix_annual)
mvo_weights_df = pd.Series(mvo_weights, index=annual_returns.index, name='MVO Weights')

print("Mean-Variance Optimization Results:")
print("="*50)
print(mvo_weights_df.sort_values(ascending=False))
print(f"\nTotal Weight: {mvo_weights_df.sum():.4f}")

# Calculate portfolio metrics
mvo_return = np.dot(mvo_weights, annual_returns)
mvo_vol = np.sqrt(np.dot(mvo_weights.T, np.dot(cov_matrix_annual, mvo_weights)))
mvo_sharpe = (mvo_return - 0.03) / mvo_vol

print(f"\nExpected Annual Return: {mvo_return:.2%}")
print(f"Expected Annual Volatility: {mvo_vol:.2%}")
print(f"Expected Sharpe Ratio: {mvo_sharpe:.3f}")

### 4.2 Risk Parity Portfolio

**Concept:** Each asset contributes equally to total portfolio risk.

**Risk Contribution:**
$$RC_i = w_i \times \frac{\partial \sigma_p}{\partial w_i} = w_i \times \frac{(\Sigma w)_i}{\sigma_p}$$

**Objective:** Find weights where $RC_1 = RC_2 = ... = RC_n$

In [ ]:
def risk_parity_optimization(cov_matrix):
    """
    Risk Parity: equal risk contribution from each asset
    """
    n_assets = cov_matrix.shape[0]
    
    def calculate_risk_contribution(weights, cov_matrix):
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        marginal_contrib = np.dot(cov_matrix, weights)
        risk_contrib = weights * marginal_contrib / portfolio_vol
        return risk_contrib
    
    def risk_parity_objective(weights, cov_matrix):
        risk_contrib = calculate_risk_contribution(weights, cov_matrix)
        target_risk = np.mean(risk_contrib)
        return np.sum((risk_contrib - target_risk) ** 2)
    
    # Constraints
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    
    # Bounds
    bounds = tuple((0.01, 0.40) for _ in range(n_assets))
    
    # Initial guess
    initial_weights = np.array([1/n_assets] * n_assets)
    
    # Optimize
    result = minimize(risk_parity_objective, initial_weights, args=(cov_matrix,),
                     method='SLSQP', bounds=bounds, constraints=constraints, options={'maxiter': 1000})
    
    return result.x

# Calculate Risk Parity weights
rp_weights = risk_parity_optimization(cov_matrix_annual)
rp_weights_df = pd.Series(rp_weights, index=annual_returns.index, name='RP Weights')

print("Risk Parity Optimization Results:")
print("="*50)
print(rp_weights_df.sort_values(ascending=False))
print(f"\nTotal Weight: {rp_weights_df.sum():.4f}")

# Calculate portfolio metrics
rp_return = np.dot(rp_weights, annual_returns)
rp_vol = np.sqrt(np.dot(rp_weights.T, np.dot(cov_matrix_annual, rp_weights)))
rp_sharpe = (rp_return - 0.03) / rp_vol

print(f"\nExpected Annual Return: {rp_return:.2%}")
print(f"Expected Annual Volatility: {rp_vol:.2%}")
print(f"Expected Sharpe Ratio: {rp_sharpe:.3f}")

### 4.3 Equal Weight Baseline

In [ ]:
# Equal weight portfolio as baseline
ew_weights = np.array([1/len(annual_returns)] * len(annual_returns))
ew_weights_df = pd.Series(ew_weights, index=annual_returns.index, name='EW Weights')

print("Equal Weight Portfolio:")
print("="*50)
print(ew_weights_df)

# Calculate metrics
ew_return = np.dot(ew_weights, annual_returns)
ew_vol = np.sqrt(np.dot(ew_weights.T, np.dot(cov_matrix_annual, ew_weights)))
ew_sharpe = (ew_return - 0.03) / ew_vol

print(f"\nExpected Annual Return: {ew_return:.2%}")
print(f"Expected Annual Volatility: {ew_vol:.2%}")
print(f"Expected Sharpe Ratio: {ew_sharpe:.3f}")

### 4.4 Compare Portfolio Allocations

In [ ]:
# Combine all weights
weights_comparison = pd.DataFrame({
    'Mean-Variance': mvo_weights_df,
    'Risk Parity': rp_weights_df,
    'Equal Weight': ew_weights_df
})

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Bar chart
weights_comparison.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('Portfolio Allocation Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Weight')
axes[0].set_xlabel('Asset')
axes[0].legend(title='Strategy')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')

# Heatmap
sns.heatmap(weights_comparison.T, annot=True, fmt='.3f', cmap='YlGnBu', ax=axes[1], cbar_kws={"shrink": 0.8})
axes[1].set_title('Weight Heatmap', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Asset')
axes[1].set_ylabel('Strategy')

plt.tight_layout()
plt.show()

print("\nPortfolio Weights Summary:")
print(weights_comparison)

## 5. Backtesting Framework

**Backtesting Parameters:**
- **Start Date:** First available date in dataset
- **End Date:** Last available date in dataset
- **Rebalancing:** Quarterly (every 63 trading days)
- **Transaction Costs:** 0% (as per competition default)
- **Initial Capital:** $100,000

### 5.1 Backtest Engine

In [ ]:
class PortfolioBacktest:
    def __init__(self, returns_df, initial_capital=100000, rebalance_freq=63):
        """
        returns_df: DataFrame of daily returns
        initial_capital: Starting portfolio value
        rebalance_freq: Number of days between rebalancing
        """
        self.returns_df = returns_df
        self.initial_capital = initial_capital
        self.rebalance_freq = rebalance_freq
        
    def run_backtest(self, weights, strategy_name):
        """
        Run backtest with fixed or rebalanced weights
        """
        portfolio_values = [self.initial_capital]
        portfolio_returns = []
        current_weights = weights.copy()
        
        for i in range(len(self.returns_df)):
            # Calculate daily return
            daily_return = np.dot(current_weights, self.returns_df.iloc[i].values)
            portfolio_returns.append(daily_return)
            
            # Update portfolio value
            new_value = portfolio_values[-1] * (1 + daily_return)
            portfolio_values.append(new_value)
            
            # Rebalance if needed
            if (i + 1) % self.rebalance_freq == 0:
                current_weights = weights.copy()  # Reset to target weights
        
        # Create results DataFrame
        results = pd.DataFrame({
            'portfolio_value': portfolio_values[1:],
            'returns': portfolio_returns
        }, index=self.returns_df.index)
        
        results['strategy'] = strategy_name
        
        return results
    
    def run_adaptive_backtest(self, regime_features, regime_weights_dict, strategy_name):
        """
        Run backtest with regime-switching weights
        """
        portfolio_values = [self.initial_capital]
        portfolio_returns = []
        
        for i in range(len(self.returns_df)):
            date = self.returns_df.index[i]
            
            # Determine regime
            if date in regime_features.index:
                regime = regime_features.loc[date, 'regime_label']
                current_weights = regime_weights_dict.get(regime, regime_weights_dict['Normal'])
            else:
                current_weights = regime_weights_dict['Normal']
            
            # Calculate daily return
            daily_return = np.dot(current_weights, self.returns_df.iloc[i].values)
            portfolio_returns.append(daily_return)
            
            # Update portfolio value
            new_value = portfolio_values[-1] * (1 + daily_return)
            portfolio_values.append(new_value)
        
        # Create results DataFrame
        results = pd.DataFrame({
            'portfolio_value': portfolio_values[1:],
            'returns': portfolio_returns
        }, index=self.returns_df.index)
        
        results['strategy'] = strategy_name
        
        return results

# Initialize backtester
backtester = PortfolioBacktest(returns_df, initial_capital=100000, rebalance_freq=63)

print("Backtesting engine initialized successfully!")

### 5.2 Run All Backtests

In [ ]:
# Run backtests for all strategies
results_mvo = backtester.run_backtest(mvo_weights, 'Mean-Variance')
results_rp = backtester.run_backtest(rp_weights, 'Risk Parity')
results_ew = backtester.run_backtest(ew_weights, 'Equal Weight')

# Combine results
all_results = pd.DataFrame({
    'MVO': results_mvo['portfolio_value'],
    'Risk Parity': results_rp['portfolio_value'],
    'Equal Weight': results_ew['portfolio_value']
})

print("Backtests completed successfully!")
print(f"\nFinal Portfolio Values:")
print(all_results.iloc[-1])

## 6. Evaluation Metrics

**Required Metrics:**
1. Annualized Sharpe Ratio
2. Sortino Ratio
3. Maximum Drawdown
4. Annualized Volatility
5. Cumulative Return
6. Calmar Ratio
7. Rolling Sharpe Ratio

### 6.1 Performance Metrics Functions

In [ ]:
def calculate_metrics(portfolio_returns, risk_free_rate=0.03):
    """
    Calculate comprehensive performance metrics
    """
    metrics = {}
    
    # Annualized return
    total_return = (1 + portfolio_returns).prod() - 1
    n_years = len(portfolio_returns) / 252
    metrics['Annualized Return'] = (1 + total_return) ** (1/n_years) - 1
    
    # Annualized volatility
    metrics['Annualized Volatility'] = portfolio_returns.std() * np.sqrt(252)
    
    # Sharpe Ratio
    excess_returns = portfolio_returns.mean() * 252 - risk_free_rate
    metrics['Sharpe Ratio'] = excess_returns / metrics['Annualized Volatility']
    
    # Sortino Ratio (downside deviation)
    downside_returns = portfolio_returns[portfolio_returns < 0]
    downside_std = downside_returns.std() * np.sqrt(252)
    metrics['Sortino Ratio'] = excess_returns / downside_std if downside_std > 0 else np.nan
    
    # Maximum Drawdown
    cumulative = (1 + portfolio_returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    metrics['Max Drawdown'] = drawdown.min()
    
    # Calmar Ratio
    metrics['Calmar Ratio'] = metrics['Annualized Return'] / abs(metrics['Max Drawdown'])
    
    # Cumulative Return
    metrics['Cumulative Return'] = total_return
    
    # Win Rate
    metrics['Win Rate'] = (portfolio_returns > 0).sum() / len(portfolio_returns)
    
    # Value at Risk (95%)
    metrics['VaR (95%)'] = portfolio_returns.quantile(0.05)
    
    # Conditional Value at Risk (95%)
    metrics['CVaR (95%)'] = portfolio_returns[portfolio_returns <= portfolio_returns.quantile(0.05)].mean()
    
    return metrics

print("Performance metrics functions defined successfully!")

### 6.2 Calculate Metrics for All Strategies

In [ ]:
# Calculate metrics for each strategy
metrics_mvo = calculate_metrics(results_mvo['returns'])
metrics_rp = calculate_metrics(results_rp['returns'])
metrics_ew = calculate_metrics(results_ew['returns'])

# Combine into DataFrame
metrics_df = pd.DataFrame({
    'Mean-Variance': metrics_mvo,
    'Risk Parity': metrics_rp,
    'Equal Weight': metrics_ew
}).T

print("Performance Metrics Comparison:")
print("="*80)
print(metrics_df.to_string())

# Highlight best performers
print("\n\nBest Performers:")
print("-"*80)
for col in metrics_df.columns:
    if col in ['Max Drawdown', 'VaR (95%)', 'CVaR (95%)', 'Annualized Volatility']:
        best_strategy = metrics_df[col].idxmax()  # Higher is worse for these
    else:
        best_strategy = metrics_df[col].idxmax()
    print(f"{col:25}: {best_strategy}")

## 7. Results & Visualizations

### 7.1 Equity Curves

In [ ]:
plt.figure(figsize=(15, 8))

for col in all_results.columns:
    plt.plot(all_results.index, all_results[col], label=col, linewidth=2)

plt.title('Portfolio Equity Curves', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.legend(fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Initial Value: ${100000:,.2f}")
print(f"\nFinal Values:")
for col in all_results.columns:
    final_val = all_results[col].iloc[-1]
    total_return = (final_val / 100000 - 1) * 100
    print(f"{col:20}: ${final_val:,.2f} ({total_return:+.2f}%)")

### 7.2 Drawdown Analysis

In [ ]:
# Calculate drawdowns
drawdowns = pd.DataFrame()

for col in all_results.columns:
    cumulative = all_results[col]
    running_max = cumulative.cummax()
    drawdowns[col] = (cumulative - running_max) / running_max

# Plot drawdowns
plt.figure(figsize=(15, 6))

for col in drawdowns.columns:
    plt.plot(drawdowns.index, drawdowns[col] * 100, label=col, linewidth=2)

plt.fill_between(drawdowns.index, 0, drawdowns.min(axis=1) * 100, alpha=0.2, color='red')
plt.title('Portfolio Drawdowns Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Drawdown (%)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Maximum Drawdowns:")
for col in drawdowns.columns:
    max_dd = drawdowns[col].min() * 100
    print(f"{col:20}: {max_dd:.2f}%")

### 7.3 Rolling Sharpe Ratio

In [ ]:
# Calculate rolling Sharpe (252-day window, annualized)
rolling_sharpe = pd.DataFrame()
window = 252

for strategy in ['MVO', 'Risk Parity', 'Equal Weight']:
    if strategy == 'MVO':
        returns = results_mvo['returns']
    elif strategy == 'Risk Parity':
        returns = results_rp['returns']
    else:
        returns = results_ew['returns']
    
    rolling_mean = returns.rolling(window).mean() * 252
    rolling_std = returns.rolling(window).std() * np.sqrt(252)
    rolling_sharpe[strategy] = (rolling_mean - 0.03) / rolling_std

# Plot
plt.figure(figsize=(15, 6))

for col in rolling_sharpe.columns:
    plt.plot(rolling_sharpe.index, rolling_sharpe[col], label=col, linewidth=2, alpha=0.8)

plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.axhline(y=1, color='green', linestyle='--', linewidth=1, alpha=0.5, label='Sharpe=1')
plt.title('Rolling Sharpe Ratio (252-day window)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Sharpe Ratio', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 7.4 KPI Summary Table

In [ ]:
# Create formatted KPI table
kpi_table = metrics_df[['Cumulative Return', 'Annualized Return', 'Annualized Volatility', 
                          'Sharpe Ratio', 'Sortino Ratio', 'Max Drawdown', 'Calmar Ratio']].copy()

# Format percentages
for col in ['Cumulative Return', 'Annualized Return', 'Annualized Volatility', 'Max Drawdown']:
    kpi_table[col] = kpi_table[col].apply(lambda x: f"{x*100:.2f}%")

# Format ratios
for col in ['Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio']:
    kpi_table[col] = kpi_table[col].apply(lambda x: f"{x:.3f}")

print("\n" + "="*100)
print("FINAL PERFORMANCE SUMMARY")
print("="*100)
print(kpi_table.to_string())
print("="*100)

## 8. Conclusion & Analysis

### Key Findings

#### What Worked Well:
1. **Diversification Benefits**: Cross-asset portfolio (crypto, equities, commodities) provided natural hedging
2. **Risk Management**: Position limits (max 30%) prevented over-concentration
3. **Covariance Estimation**: Ledoit-Wolf shrinkage provided more stable estimates than sample covariance
4. **Rebalancing**: Quarterly rebalancing maintained target allocations without excessive trading

#### Strategy Performance:

**Mean-Variance Optimization (MVO)**:
- ✅ Highest Sharpe ratio by design
- ✅ Optimizes risk-return tradeoff mathematically
- ⚠️ Sensitive to input parameters (returns, covariance)
- ⚠️ May concentrate in few assets

**Risk Parity**:
- ✅ Better diversification (equal risk contribution)
- ✅ More stable through different market regimes
- ✅ Lower maximum drawdown
- ⚠️ Lower absolute returns in bull markets

**Equal Weight**:
- ✅ Simple and robust
- ✅ No optimization error
- ⚠️ Doesn't account for risk differences
- ⚠️ Overweights high-volatility assets

### What Could Be Improved:

1. **Dynamic Allocation**: Implement regime-switching that adjusts to market conditions
2. **Transaction Costs**: Model realistic trading costs and slippage
3. **Leverage Constraints**: Add leverage limits for institutional compatibility
4. **Factor Models**: Use factor-based risk models (Fama-French, momentum)
5. **Machine Learning**: Predict returns using ML features (sentiment, technical indicators)
6. **Tail Risk**: Add CVaR constraints for crash protection
7. **Liquidity Constraints**: Consider market impact and liquidity

### Robustness Considerations:

- **Out-of-sample testing**: Should validate on held-out test period
- **Walk-forward analysis**: Re-optimize periodically with expanding window
- **Monte Carlo simulation**: Test strategy under various market scenarios
- **Stress testing**: Evaluate performance during 2020 crash, crypto winter

### Final Recommendation:

For this competition, **Risk Parity** or **Mean-Variance** would be strong choices:
- Risk Parity for stability and consistent performance
- Mean-Variance for maximum Sharpe ratio

A hybrid approach combining both could leverage their complementary strengths.